# 处理计算

在以 Azure 机器学习试验形式运行脚本时，需要定义试验运行的执行上下文。执行上下文由以下内容组成：

* 脚本的 Python 环境，其中必须包括脚本中使用的所有 Python 包。
* 将在其上运行脚本的计算目标。该目标可以是启动试验运行的本地工作站，也可以是按需预配的远程计算目标，例如训练群集。

在该笔记本中，你将探索试验的环境和计算目标。

## 连接到工作区

首先，请连接到你的工作区。

> **备注**：如果尚未与 Azure 订阅建立经过身份验证的会话，则系统将提示你通过执行以下操作进行身份验证：单击链接，输入验证码，然后登录到 Azure。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 准备试验数据

在该笔记本中，你将使用包含糖尿病患者详细信息的数据集。运行以下单元格以创建此数据集（如果已有此数据集，则代码将查找现有版本）

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## 创建训练脚本

运行以下两个单元格，创建以下内容：

1. 用于新试验的文件夹
2. 使用 **scikit-learn** 训练模型并使用 **matplotlib** 绘制 ROC 曲线的训练脚本文件。

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_logistic'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Get script arguments
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# Set regularization hyperparameter
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## 定义环境

在 Azure 机器学习中以试验形式运行 Python 脚本时，系统将创建 Conda 环境，用于定义脚本的执行上下文。Azure 机器学习提供了一个默认环境，其中包括许多常见包；例如，**azureml-defaults** 包，其中包含处理试验运行所需的库，以及 **pandas** 和 **numpy** 等常用包。

你还可以在 Conda 规范文件中定义自己的环境，并使用 **conda** 或 **pip** 添加包，以确保试验可以访问其所需的所有库。

> **备注**：首先安装 conda 依赖项，然后安装 **pip** 依赖项。由于安装 pip 依赖项需要 pip 包，因此最好将其包含在 conda 依赖项中。

运行以下单元格以在与此笔记本相同的文件夹中创建名为 *Experiment_env.yml* 的 Conda 规范文件。

In [ ]:
%%writefile $experiment_folder/experiment_env.yml
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow

现在你可以使用自定义 conda 规范文件为试验创建一个环境

In [ ]:
from azureml.core import Environment

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/experiment_env.yml")

# Let Azure ML manage dependencies
experiment_env.python.user_managed_dependencies = False 

# Print the environment details
print(experiment_env.name, 'defined.')
print(experiment_env.python.conda_dependencies.serialize_to_string())

现在，你可以使用环境以试验方式运行脚本。

以下代码会将你创建的环境分配给 ScriptRunConfig，并提交试验。试验运行期间，注意小组件和 **azureml_logs/60_control_log.txt** 输出日志中的运行详细信息，你将看到正在构建 conda 环境。

> **备注**：下面的代码为脚本运行创建了 **DockerConfiguration**，并将其 **use_docker** 属性设置为 **True**，以便将脚本的环境托管在 Docker 容器中。这是默认行为，因此可省略该行为，但这里显式包含此行为。

In [ ]:
from azureml.core import Experiment, ScriptRunConfig
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                             '--input-data', diabetes_ds.as_named_input('training_data')], # Reference to dataset
                                environment=experiment_env,
                                docker_runtime_config=DockerConfiguration(use_docker=True)) # Use docker to host environment

# submit the experiment
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

试验成功使用了环境，其中包括其所需的所有包 - 你可以根据 Azure 机器学习工作室中运行的试验或通过运行以下代码来查看指标和输出 - 包括使用 **scikit-learn** 训练的模型和使用 **matplotlib** 生成的 ROC 图表图像。

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## 注册环境

使用所需包定义环境后，可以在工作区中注册它。

In [ ]:
# Register the environment
experiment_env.register(workspace=ws)

请注意，该环境的注册名称为首次创建时所分配的名称（本例中为 *diabetes-experiment-env*）。

注册环境后，可以将其重用于需求相同的任何脚本。例如，创建文件夹和脚本以使用其他算法训练糖尿病模型：

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_tree'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Get script arguments
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

现在，可以检索已注册的环境并在运行替代训练脚本的新试验中使用该环境（这次没有正则化参数，因为决策树分类器不需要该参数）。

In [ ]:
# get the registered environment
registered_env = Environment.get(ws, 'experiment_env')

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                              script='diabetes_training.py',
                              arguments = ['--input-data', diabetes_ds.as_named_input('training_data')], # Reference to dataset
                              environment=registered_env,
                              docker_runtime_config=DockerConfiguration(use_docker=True)) # Use docker to host environment 

# submit the experiment
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

此时试验运行速度更快，这是因为上次运行中缓存有匹配环境，因此无需在本地计算机上重建。但即使是在不同的计算目标上，也会创建和使用相同的环境 - 确保试验脚本执行上下文的一致性。

接下来看一下试验的指标和输出。

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## 查看已注册的环境

除注册自己的环境以外，你还可以将预建的“策展”环境用于常见类型的试验。以下代码列出了所有已注册的环境：

In [ ]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)
for env in envs:
    print("Name",env)

所有策展环境的名称都以 ***AzureML-*** 开头（不能将此前缀用于自己的环境）。

接下来更深入地了解策展环境，了解每个环境中包含的包。

In [ ]:
for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

## 创建计算群集

许多情况下，本地计算资源可能不足，无法进行需处理大量数据的复杂试验或长期运行的试验；并且你可能想利用在云中动态创建和使用计算资源这一功能。Azure 机器学习支持一系列计算目标，你可以在工作区中定义这些目标并将其用于运行试验；只需在使用资源时为其付费。

可以在 [Azure 机器学习工作室](https://ml.azure.com)中创建计算群集，或者使用 Azure 机器学习 SDK 创建。以下代码单元检查工作区中是否存在具有指定名称的计算群集，如果不存在，则创建它。

> **重要提示**：在运行计算群集之前，在下面的代码中将 *your-compute-cluster* 更改为适合你的计算群集的名称，如果有，则可以指定现有群集的名称。群集名称必须是长度在 2 到 16 个字符之间的全局唯一名称。有效字符是字母、数字和 - 字符。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

> **备注**：计算实例和群集是基于标准的 Azure 虚拟机映像。对于本练习，建议使用 *Standard_DS11_v2* 映像来实现成本和性能的最佳平衡。如果你的订阅配额不包含此映像，请选择其他映像；但请记住，较大的映像可能会产生更高的成本，但较小的映像可能不够完成任务。或者，可以请 Azure 管理员扩展你的配额。

## 在远程计算上运行试验

现在即可再次运行之前运行的试验，不过这次是在你创建的计算群集上运行。 

> **备注**：该试验耗时更长，这是因为容器映像必须在 conda 环境中构建，然后必须先启动群集节点并部署映像才能运行脚本。对于糖尿病训练脚本等简单试验，这似乎效率不高；但假设需运行的是耗时数小时的更复杂的试验 - 动态创建可缩放性更高的计算可能会显著减少总时长。

In [ ]:
# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--input-data', diabetes_ds.as_named_input('training_data')],
                                environment=registered_env,
                                compute_target=cluster_name) 

# submit the experiment
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()

等待试验运行时，可以在上方的小组件中或 [Azure 机器学习工作室](https://ml.azure.com)中检查计算状态。你还可以使用以下代码检查计算状态。

In [ ]:
cluster_state = training_cluster.get_status()
print(cluster_state.allocation_state, cluster_state.current_node_count)

请注意，状态从“*稳定*”变为“*调整中*”需要一段时间（此时非常适合休息喝咖啡！）。若要在运行结束之前阻止内核，请运行以下单元格。

In [ ]:
run.wait_for_completion()

请注意页面右上方的内核指示器，当它从 **&#9899;** 变为 **&#9711;** 时，表明代码已运行完毕。

试验完成后，可以获取试验运行所生成的指标和文件。此时，文件将包含用于构建映像和管理计算的日志。

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

现在即可注册由试验训练的模型。

In [ ]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Compute cluster'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **详细信息**：
>
> - 有关 Azure 机器学习中的环境的详细信息，请参阅[在 Azure 机器学习中创建和使用软件环境](https://docs.microsoft.com/azure/machine-learning/how-to-use-environments)
> - 有关 Azure 机器学习中的计算目标的详细信息，请参阅 [Azure 机器学习中的计算目标是什么？](https://docs.microsoft.com/azure/machine-learning/concept-compute-target)